In [4]:
import pandas as pd
import random
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import xgboost as xgb
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV
import numpy as np
from sklearn.metrics import classification_report

In [2]:
humanset1 = pd.read_json("set1_human.json")
humanset2 = pd.read_json("set2_human.json")
machineset1 = pd.read_json("set1_machine.json")
machineset1 = machineset1.drop("machine_id",axis=1)
machineset2 = pd.read_json("set2_machine.json")
machineset2 = machineset2.drop("machine_id",axis=1)
test = pd.read_json("test.json")
test["txt2"] = test["txt"].apply(lambda x:[str(each) for each in x])
test["txt2"] = test["txt2"].apply(lambda x:" ".join(x))

In [3]:
key = [1]*humanset1.shape[0]
humanset1["key"] = key
key = [1]*humanset2.shape[0]
humanset2["key"] = key
key = [0]*machineset1.shape[0]
machineset1["key"] = key
key = [0]*machineset2.shape[0]
machineset2["key"] = key
humanset = pd.concat([humanset1,humanset2], axis = 0)
machineset = pd.concat([machineset1,machineset2], axis = 0)
data = pd.concat([humanset, machineset], axis = 0)
data["txt2"] = data["txt"].apply(lambda x:[str(each) for each in x])
data["txt2"] = data["txt2"].apply(lambda x:" ".join(x))
data['length'] = data['txt'].apply(lambda x: str(len(x)))

In [4]:
response = pd.concat([data["txt2"], data["length"]], axis=1)
pred = y = data["key"]

In [5]:
X_train, X_test, y_train, y_test = train_test_split(data['txt2'], data['key'], train_size=0.8, random_state=5)
vectorizer = CountVectorizer()
vectorizer.fit(X_train)

X = vectorizer.transform(X_train)
X = X.toarray()
Xt = vectorizer.transform(X_test)
Xt = Xt.toarray()

In [6]:
k = 5
smote = SMOTE(sampling_strategy='auto', k_neighbors=k, random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y_train)

In [30]:
xgb_model = xgb.XGBClassifier(max_depth=20, n_estimators=600, learning_rate=0.1, random_state=42)
xgb_model.fit(X_resampled, y_resampled)
pred = xgb_model.predict(Xt)

In [31]:
from sklearn.metrics import classification_report
print(classification_report(y_true=y_test, y_pred=pred, digits=10))

              precision    recall  f1-score   support

           0  0.8475609756 0.5574866310 0.6725806452       748
           1  0.9866666667 0.9969473727 0.9917803782     24569

    accuracy                      0.9839633448     25317
   macro avg  0.9171138211 0.7772170019 0.8321805117     25317
weighted avg  0.9825567383 0.9839633448 0.9823495056     25317



RF

In [7]:
model =  LogisticRegression(C=0.004832930238571752, solver='liblinear') 
model.fit(X_resampled, y_resampled)
pred = model.predict(Xt)
from sklearn.metrics import classification_report
print(classification_report(y_true=y_test, y_pred=pred, digits=10))

              precision    recall  f1-score   support

           0  0.7582205029 0.5240641711 0.6197628458       748
           1  0.9856451613 0.9949122878 0.9902570439     24569

    accuracy                      0.9810009085     25317
   macro avg  0.8719328321 0.7594882295 0.8050099449     25317
weighted avg  0.9789258168 0.9810009085 0.9793106577     25317



In [8]:
test = pd.read_json("test.json")
test["txt2"] = test["txt"].apply(lambda x:[str(each) for each in x])
test["txt2"] = test["txt2"].apply(lambda x:" ".join(x))
X_test = vectorizer.transform(test['txt2'])
X_test = X_test.toarray()
result = model.predict(X_test)
a = list(range(1000))
result_df = pd.DataFrame({
    'id' : a,
    'Predicted' : result,
})
result_df.to_csv('result3.csv', index=False)

In [33]:
c=0
for i in result:
    c=c+i
c

478

In [44]:
df1 = pd.read_csv('result1.csv', index_col='id')
df2 = pd.read_csv('result2.csv', index_col='id')
merged = pd.merge(df1, df2, on='id', suffixes=('_left', '_right'))
diffs = merged.query('Predicted_left != Predicted_right')
diffs

,Predicted_left,Predicted_right
id,,
1,0,1
2,0,1
3,0,1
4,0,1
7,0,1
...,...,...
959,0,1
965,1,0
972,0,1


In [ ]:
#Hyperparamter
hyperpara = {
    'penalty':['l1','l2'],
    'C':np.logspace(-4,4,20),
    'solver':['liblinear']
}
model = LogisticRegression()

grid_search = GridSearchCV(
    estimator=model,
    param_grid=hyperpara,
    scoring='f1_macro',
    n_jobs = 2,
    cv=5,
    verbose=True
)

grid_result = grid_search.fit(X_resampled, y_resampled)
grid_search.best_params
pred = grid_search.predict(Xt)
best = grid_search.best_estimator
print(best)

Fitting 5 folds for each of 40 candidates, totalling 200 fits
